In [1]:
import os
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from time import time

In [2]:
def breaker(num: int=50, char: str="*") -> None:
    print("\n" + num*char + "\n")


class CFG(object):
    def __init__(self,
         seed: int = 42,
         n_splits: int = 5,
         show_info: bool = False,
         ):

        self.seed = seed
        self.n_splits = n_splits
        self.show_info = show_info
        self.tr_data = "../input/2022-regression-data-challenge/train.csv"
        self.ts_data = "../input/2022-regression-data-challenge/test.csv"
        self.ss_data = "../input/2022-regression-data-challenge/submission.csv"
        self.model_save_path = "models"
        if not os.path.exists(self.model_save_path): os.makedirs(self.model_save_path)

cfg = CFG(seed=42, show_info=True)
sc_y = StandardScaler()    

In [3]:
class Model(object):
    def __init__(self, model_name: str, preprocessor, seed: int):
        self.model_name = model_name

        if self.model_name == "lr":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", LinearRegression()),
                ]
            )
        
        elif self.model_name == "knr":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", KNeighborsRegressor()),
                ]
            )
        
        elif self.model_name == "dtr":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", DecisionTreeRegressor(random_state=seed)),
                ]
            )

        elif self.model_name == "etr":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", ExtraTreeRegressor(random_state=seed)),
                ]
            )
        
        elif self.model_name == "rfr":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", RandomForestRegressor(random_state=seed)),
                ]
            )
        
        elif self.model_name == "gbr":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", GradientBoostingRegressor(random_state=seed)),
                ]
            )
        
        elif self.model_name == "abr":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", AdaBoostRegressor(random_state=seed)),
                ]
            )
        
        elif self.model_name == "etrs":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", ExtraTreesRegressor(random_state=seed)),
                ]
            )
        
        elif self.model_name == "svr":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", SVR()),
                ]
            )
        
        elif self.model_name == "xgbr":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", XGBRegressor(random_state=seed)),
                ]
            )

In [4]:
def get_scores(y_true: np.ndarray, y_pred: np.ndarray) -> tuple:
    return mean_absolute_error(y_true, y_pred), \
           mean_squared_error(y_true, y_pred), \
           r2_score(y_true, y_pred)


def print_scores(mae: float, mse: float, r2: np.ndarray) -> None:
    print(f"MAE : {mae:.5f}")
    print(f"MSE : {mse:.5f}")
    print(f"R2  : {mse:.5f}")

In [5]:
names: list = ["lr", "knr", "svr", "dtr", "etr", "abr", "gbr", "etrs", "rfr", "xgbr"]
# names: list = ["lr", "knr"]

tr_df = pd.read_csv(cfg.tr_data)
tr_df = tr_df.drop(columns=["Id"])

X = tr_df.iloc[:, 1:].copy().values
y = tr_df.iloc[:, 0].copy().values

features = [i for i in range(X.shape[1])]

feature_transformer = Pipeline(
    steps=[
        ("Standard_Scaler", StandardScaler())
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("features", feature_transformer, features),
    ]
)

best_mse: float = np.inf
for name in names:
    fold = 1
    if cfg.show_info: breaker()
    for tr_idx, va_idx in KFold(n_splits=cfg.n_splits, random_state=cfg.seed, shuffle=True).split(X):
        X_train, X_valid, y_train, y_valid = X[tr_idx], X[va_idx], y[tr_idx], y[va_idx]
        
        y_train = sc_y.fit_transform(y_train.reshape(-1, 1)).reshape(-1)
        y_valid = sc_y.transform(y_valid.reshape(-1, 1)).reshape(-1)
        
        my_pipeline = Model(name, preprocessor, cfg.seed)
        my_pipeline.model.fit(X_train, y_train)

        y_pred = my_pipeline.model.predict(X_valid)
        if cfg.show_info:
            mae, mse, r2 = get_scores(y_valid, y_pred)
            print(f"{my_pipeline.model_name}, {fold}\n")
            print_scores(mae, mse, r2)
            print("")
        else:
            _, mse, _ = get_scores(y_valid, y_pred)

        if mse < best_mse:
            best_mse = mse
            model_fold_name = f"{name}_{fold}"

            with open(os.path.join(cfg.model_save_path, f"best_model.pkl"), "wb") as fp: pickle.dump(my_pipeline.model, fp)
        
        with open(os.path.join(cfg.model_save_path, f"{name}_{fold}.pkl"), "wb") as fp: pickle.dump(my_pipeline.model, fp)
        
        fold += 1
    

if cfg.show_info: 
    breaker()
    print(f"Best Model : {model_fold_name.split('_')[0]}, Best Fold : {model_fold_name.split('_')[1]}")

breaker()


**************************************************

lr, 1

MAE : 0.81464
MSE : 0.97776
R2  : 0.97776

lr, 2

MAE : 0.81001
MSE : 0.97975
R2  : 0.97975

lr, 3

MAE : 0.76822
MSE : 0.88663
R2  : 0.88663

lr, 4

MAE : 0.78061
MSE : 0.91306
R2  : 0.91306

lr, 5

MAE : 0.73607
MSE : 0.80464
R2  : 0.80464


**************************************************

knr, 1

MAE : 0.86618
MSE : 1.15446
R2  : 1.15446

knr, 2

MAE : 0.85396
MSE : 1.14151
R2  : 1.14151

knr, 3

MAE : 0.80536
MSE : 1.00809
R2  : 1.00809

knr, 4

MAE : 0.79871
MSE : 1.01177
R2  : 1.01177

knr, 5

MAE : 0.78543
MSE : 0.95320
R2  : 0.95320


**************************************************

svr, 1

MAE : 0.76131
MSE : 0.92352
R2  : 0.92352

svr, 2

MAE : 0.75109
MSE : 0.92608
R2  : 0.92608

svr, 3

MAE : 0.71983
MSE : 0.85488
R2  : 0.85488

svr, 4

MAE : 0.72194
MSE : 0.85579
R2  : 0.85579

svr, 5

MAE : 0.69179
MSE : 0.77094
R2  : 0.77094


**************************************************

dtr, 1

MAE : 0.91467
MSE : 